In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.filterwarnings("ignore")
import os
import zipfile
import pytesseract
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay

/kaggle/input/donorsearch/images/444___7820ca3c69a44d98ae043a6b33dc6201.png
/kaggle/input/donorsearch/images/405___c42f4f27f9a8476e933ca80f37fb66d7.png
/kaggle/input/donorsearch/images/444___c0bbd4a7dc3a47968c7430e43d534661.png
/kaggle/input/donorsearch/images/405___aa736a930da8469b8091617346fd81cd.png
/kaggle/input/donorsearch/images/448___621433877a3643969f3c702947e56527.png
/kaggle/input/donorsearch/images/448___270ec991cd3b4652a7558d992084b4ec.png
/kaggle/input/donorsearch/images/402___fae1da9d75d54e24bcf24955ecf739a2.png
/kaggle/input/donorsearch/images/444___c50d8219ae7d4f3699b5ba4e8f05affa.png
/kaggle/input/donorsearch/images/444___d0b692b4e9734742a57644e8f7142c4d.png
/kaggle/input/donorsearch/images/405___a98cdd5fdc8945f983aab3a33fb86bad.png
/kaggle/input/donorsearch/images/402___335f83b3a37b403fb31088c8132f74b4.png
/kaggle/input/donorsearch/images/448___44e7977a414b40c0936ad4112b573a62.png
/kaggle/input/donorsearch/images/402___136d64ae8d174354bc2af057d77f93a7.png
/kaggle/inpu

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ace076a3310>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [2]:
# Путь к папке с изображениями и имя выходного CSV файла
folder_path = '/kaggle/input/donorsearch/images'  # Замените на путь к вашей папке с изображениями
output_csv = 'dataset.csv'

def create_dataset(folder_path, output_csv):
    data = []

    # Проходим по всем файлам в указанной папке
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        
        # Проверяем, что это файл изображения
        if os.path.isfile(img_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            label = os.path.splitext(filename)[0] 
            data.append({'image_path': img_path, 'label': label})

    # Создаем DataFrame и сохраняем в CSV
    pd.DataFrame(data).to_csv(output_csv, index=False)

# Создаем датасет
create_dataset(folder_path, output_csv)

In [3]:
med_cert = pd.read_csv('dataset.csv')

In [4]:
med_cert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  173 non-null    object
 1   label       173 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB


In [5]:
med_cert.head()

,image_path,label
0,/kaggle/input/donorsearch/images/444___7820ca3...,444___7820ca3c69a44d98ae043a6b33dc6201
1,/kaggle/input/donorsearch/images/405___c42f4f2...,405___c42f4f27f9a8476e933ca80f37fb66d7
2,/kaggle/input/donorsearch/images/444___c0bbd4a...,444___c0bbd4a7dc3a47968c7430e43d534661
3,/kaggle/input/donorsearch/images/405___aa736a9...,405___aa736a930da8469b8091617346fd81cd
4,/kaggle/input/donorsearch/images/448___6214338...,448___621433877a3643969f3c702947e56527


In [6]:
#создадим колонку с номерами справок
def cat(file_name):
    return ''.join(filter(str.isdigit, file_name))[:3]

med_cert['category'] = med_cert['image_path'].apply(cat)
med_cert.head()

,image_path,label,category
0,/kaggle/input/donorsearch/images/444___7820ca3...,444___7820ca3c69a44d98ae043a6b33dc6201,444
1,/kaggle/input/donorsearch/images/405___c42f4f2...,405___c42f4f27f9a8476e933ca80f37fb66d7,405
2,/kaggle/input/donorsearch/images/444___c0bbd4a...,444___c0bbd4a7dc3a47968c7430e43d534661,444
3,/kaggle/input/donorsearch/images/405___aa736a9...,405___aa736a930da8469b8091617346fd81cd,405
4,/kaggle/input/donorsearch/images/448___6214338...,448___621433877a3643969f3c702947e56527,448


In [7]:
med_cert['category'].value_counts() #посмотрим на количество каждой категории

category
444    74
405    48
448    41
402     9
406     1
Name: count, dtype: int64

В датасете всего 173 строки, из которых только 48 со справками формы №405. Оставим только нужные нам и повернем каждое изображение на 90, 180 и 270 градусов.

In [8]:
source_folder = '/kaggle/input/donorsearch/images'
destination_folder = '/kaggle/working/filtered_images'  # Папка для хранения отфильтрованных изображений

# Создаем папку для хранения отфильтрованных изображений
os.makedirs(destination_folder, exist_ok=True)

# Проходим по всем файлам в папке
for filename in os.listdir(source_folder):
    # Проверяем, начинается ли имя файла с '405'
    if filename.startswith('405'):
        # Полный путь к файлу
        source_file = os.path.join(source_folder, filename)
        # Копируем файл в папку с отфильтрованными изображениями
        shutil.copy(source_file, destination_folder)

In [9]:
def create_augmented_dataset(input_dir, output_dir):
   
    # Создаем выходную директорию
    os.makedirs(output_dir, exist_ok=True)

    # Получаем список всех изображений в директории
    images = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for img_file in images:
        img_path = os.path.join(input_dir, img_file)

        # Открываем изображение
        img = Image.open(img_path)

        # Преобразуем изображение в режим RGB
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Сохраняем оригинал
        img.save(os.path.join(output_dir, img_file))  # Оригинал с меткой 0° (или "original")

        # Поворачиваем и сохраняем новые изображения
        for angle, label in zip([90, 180, 270], ['90', '180', '270']):
            rotated_img = img.rotate(angle)
            rotated_img.save(os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_rotated_{label}.png"))

# Задайте пути к директориям
input_directory = '/kaggle/working/filtered_images'  
output_directory = '/kaggle/working/more_images'  

# Создаем увеличенный датасет
create_augmented_dataset(input_directory, output_directory)

In [10]:
# Укажите путь к папке с изображениями
images_more = '/kaggle/working/more_images' #путь к папке, увеличенный вариант после поворота изображений (output_directory)
images_m = []

# Проход по всем файлам в папке
for filename in os.listdir(images_more):
    if filename.endswith(('.png')):  # Поддерживаемые форматы изображений
        # Полный путь к изображению
        image_path = os.path.join(images_more, filename)
        
        # Извлечение метки из имени файла
        label = filename.split('_')[-1].split('.')[0]  # Предполагаем, что метка после последнего '_'

        # Добавляем путь и метку в список
        images_m.append((filename, image_path, label))

# Создание DataFrame и сохранение в CSV
df = pd.DataFrame(images_m, columns=['image_path', 'label','category'])
df.to_csv('labels_image.csv', index=False)